
**Hello this is my first project --> if you have any comment please comment:)
สวัสดีครับ นี่คือโปรเจคแรกของผม ------> ถ้าคุณมีคำถามก็คอมเม้นมาได้เลยครับ :)**

สำหรับ โปรเจคแรกของผมคือ chatbot ที่ใช้สำหรับตอบคำถามของเราเอง หรือ personal assistance ของเราเองซึ่งก็อยากให้มันจำส่งที่เราบอกได้ และ สามารถ เทรน dataset ไปได้ โดยสำหรับ Prototype แรก ก็ใช้ dataset ของ hugging face มาก่อน

AutoTokenizer ใช้สำหรับแปลงข้อความเป็น token

AutoModelForCausalLM โหลดโมเดลสำหรับงาน "สร้างข้อความ"

device_map="auto" ให้โมเดลจัดการกับ GPU เอง

load_in_8bit=True ใช้ quantization เพื่อลดหน่วยความจำ (ผ่าน bitsandbytes)

In [ ]:
!pip install --upgrade pip
!pip install transformers==4.36.2 peft==0.5.0 datasets

import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"


In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="microsoft/phi-2", local_dir="./phi2", local_dir_use_symlinks=False)

In [ ]:
### โมเดลที่เราจะใช้คือ microsoft/phi-2 — ขนาดเล็ก ฉลาด และใช้ทรัพยากรไม่เยอะ เหมาะกับ Chatbot ##

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "./phi2"  # โหลดจาก local path

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ส่งโมเดลไปที่ GPU (ถ้ามี) หรือ CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
## import json สร้าง ข้อมูลมา ขำขำก่อน
import json

sample_data = [
    {"instruction": "แนะนำตัวเอง", "output": "สวัสดีครับ ผมคือบอทที่ถูกฝึกมาให้ตอบคำถามภาษาไทย!"},
    {"instruction": "ภาษา Python คืออะไร", "output": "Python เป็นภาษาที่อ่านง่าย เหมาะกับผู้เริ่มต้น..."},
    {"instruction": "วิธีชงกาแฟดำ", "output": "ต้มน้ำให้ร้อน เทน้ำผ่านกาแฟบดโดยใช้เครื่องดริปหรือกาแฟฟิลเตอร์"}
]

with open("train_data.jsonl", "w") as f:
    for item in sample_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [ ]:
import json
from datasets import Dataset

# โหลดจากไฟล์ JSONL เป็น list
with open("train_data.jsonl", "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f]

# แปลงเป็น HuggingFace Dataset
dataset = Dataset.from_list(raw_data)

# ฟังก์ชันรวม prompt + response เป็นข้อความเดียว
def format_prompt(example):
    return f"""### คำถาม:
{example['instruction']}

### คำตอบ:
{example['output']}"""

# ถ้า tokenizer ไม่มี pad_token ให้เพิ่มเข้าไป
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))  # สำคัญ! ให้โมเดลรู้ว่ามี token เพิ่มเข้ามา

# Tokenize ทุกตัวอย่าง
def tokenize_function(example):
    prompt = format_prompt(example)
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=256)

# แปลงข้อมูล
tokenized_dataset = dataset.map(tokenize_function)

# เตรียม data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
!pip uninstall -y peft transformers
!pip install -U "peft==0.9.0"
!pip install -U transformers==4.38.1

In [ ]:
## Check model layer ดูโมเดล layer ว่ามีอะไรบาง

for name, module in model.named_modules():
    if any(keyword in name.lower() for keyword in ["linear", "mlp", "dense", "fc"]):
        print(name)


Layer ที่เราจะใช้ในการทำ LoRA


self_attn.dense
mlp.fc1
mlp.fc2


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

## Config lora
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["self_attn.dense", "mlp.fc1", "mlp.fc2"],  # ← ตรงกับของ phi-2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
!pip install --upgrade transformers
import transformers
print(transformers.__version__)

!pip uninstall -y peft transformers
!pip uninstall -y accelerate

!pip install -U "peft==0.9.0"
!pip install -U transformers==4.38.1
!pip install -U accelerate==0.25.0


In [ ]:
import transformers, accelerate
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

In [ ]:
from transformers import TrainingArguments, Trainer

## FOR PC ##

 training_args = TrainingArguments(
     output_dir="./chatbot-lora-phi2",
     per_device_train_batch_size=2,
     num_train_epochs=3,
     logging_steps=5,
     save_steps=20,
     save_total_limit=2,
     evaluation_strategy="no",
     fp16=True,  # ถ้าไม่มี GPU รองรับ FP16 ก็เปลี่ยนเป็น False ได้
     report_to="none"
 )


## FOR MAC ##
# training_args = TrainingArguments(
#     output_dir="./chatbot-lora-phi2",
#     per_device_train_batch_size=1,
#     num_train_epochs=3,
#     logging_steps=5,
#     save_steps=20,
#     save_total_limit=2,
#     evaluation_strategy="no",
#     report_to="none"  # ปิดการ log ไปยังระบบภายนอก เช่น wandb
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
